In [1]:
import pandas as pd
from modeling import feature_engineering, load_object, model_predict, save_csv,get_best_params
import numpy as np
import reverse_geocoder as revgc
import xgboost as xgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold, KFold
pd.options.display.float_format = '{:.4f}'.format

C:\Users\Martin\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\Martin\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
training='training'
label='interest_integer'
clf=LGBMClassifier(n_jobs=-1,n_estimators=100,random_state=0)
df,columns,good_encoders,dp_table,df_tokenizer=load_object(clf.__class__.__name__+'_thesis')
dp_table

,columns,best value,encoders,time
after numeric feature selection,"[bathrooms, bedrooms, price, len_features, len...",0.6315,{},00:05:55.664182
after TOP N features,"[bathrooms, bedrooms, price, len_features, len...",0.6111,{},00:16:04.420428
after numeric features aggregated by categorical features,"[bathrooms, bedrooms, price, len_features, len...",0.5530,{},01:38:44.453739
after numeric features interactions,"[bathrooms, bedrooms, price, len_features, len...",0.5530,{},01:41:38.582742
after categorical feature encoding,"[bathrooms, bedrooms, price, len_features, len...",0.5433,{'building_id': <function CB_target_encoding a...,01:56:02.652604
after text feature tokenization plus PCA,"[bathrooms, bedrooms, price, len_features, len...",0.5430,{'building_id': <function CB_target_encoding a...,02:39:28.260602


In [3]:
import functools
import optuna
from hashlib import md5
from optuna.samplers import TPESampler


def objective(df,columns,training,label,encoder_dict,df_test_listingID,df_train_listingID,trial):
    params ={
        'n_estimators':trial.suggest_int('n_estimators', 10, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.3, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
    
    print(params)
    clf=LGBMClassifier(n_jobs=-1,random_state=0,**params)
    test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf ,encoder_dict)
    model_id=md5(str(params).encode()).hexdigest()[:6]
    model_id=clf.__class__.__name__+'_'+model_id+'_'+str(round(lloss,3))
    trial.set_user_attr('model_id',model_id)
    save_csv (model_id,test_pred,yoof,clf,df_test_listingID,df_train_listingID)
    return lloss


In [ ]:
n=100

sampler=TPESampler(seed=0)
study = optuna.create_study(direction='minimize',sampler=sampler)
fn=functools.partial(objective,df,columns,training,label,good_encoders,df_test['listing_id'],df_train['listing_id'])
study.optimize(fn, n_trials=n,show_progress_bar=True)

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



{'n_estimators': 694, 'learning_rate': 0.10167758525611789, 'lambda_l1': 0.39663564159126696, 'lambda_l2': 0.5266360183584486, 'num_leaves': 105, 'feature_fraction': 0.5965583595372332, 'bagging_fraction': 0.8229470565333281, 'bagging_freq': 5, 'min_child_samples': 92}
[I 2020-05-30 18:35:02,591] Finished trial#0 with value: 0.6673239233083488 with parameters: {'n_estimators': 694, 'learning_rate': 0.10167758525611789, 'lambda_l1': 0.39663564159126696, 'lambda_l2': 0.5266360183584486, 'num_leaves': 105, 'feature_fraction': 0.5965583595372332, 'bagging_fraction': 0.8229470565333281, 'bagging_freq': 5, 'min_child_samples': 92}. Best is trial#0 with value: 0.6673239233083488.
{'n_estimators': 80, 'learning_rate': 0.012483996761122442, 'lambda_l1': 2.8438497747055045e-06, 'lambda_l2': 0.0001990609424479233, 'num_leaves': 41, 'feature_fraction': 0.6702264438270331, 'bagging_fraction': 0.7840222805469661, 'bagging_freq': 6, 'min_child_samples': 30}
[I 2020-05-30 18:36:14,263] Finished trial#

In [ ]:
x=[{**t.params,**t.user_attrs} for t in study.get_trials()]
y=[ t.value for t in study.get_trials()]
table=pd.DataFrame(x)
table['score']=y

table.to_csv(clf.__class__.__name__+'_grid_results.csv')

In [ ]:
table

## Best_grid_params

In [ ]:
df,columns,clf,good_encoders=load_object(clf.__class__.__name__+'best_grid_params')
model_predict(df[columns],df[training],df[label], clf ,good_encoders)

In [4]:
import functools
import optuna
from hashlib import md5
from optuna.samplers import TPESampler

df= pd.read_csv('ensemble_predictions.csv')
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
columns=df.drop(columns=['target','training','listing_id']).columns
training='training'
label='target'

def objective(df,columns,training,label,df_test_listingID,df_train_listingID,trial):
    params ={
        'n_estimators':trial.suggest_int('n_estimators', 400, 450),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.04),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 100),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.7, 0.8),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 2),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
    
    print(params)
    clf=LGBMClassifier(n_jobs=-1,random_state=0,**params)
    test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf )
    model_id=md5(str(params).encode()).hexdigest()[:6]
    model_id=clf.__class__.__name__+'_'+model_id+'_'+str(round(lloss,3))+'LGBM'
    trial.set_user_attr('model_id',model_id)
    save_csv (model_id,test_pred,yoof,clf,df_test_listingID,df_train_listingID)
    return lloss
n=10
sampler=TPESampler(seed=0)
study = optuna.create_study(direction='minimize',sampler=sampler)
fn=functools.partial(objective,df,columns,training,label,df_test['listing_id'],df_train['listing_id'])
study.optimize(fn, n_trials=n,show_progress_bar=True)

x=[{**t.params,**t.user_attrs} for t in study.get_trials()]
y=[ t.value for t in study.get_trials()]
table=pd.DataFrame(x)
table['score']=y

table.to_csv(clf.__class__.__name__+'_grid_LGBM_Ensemble.csv')

C:\Users\Martin\anaconda3\lib\site-packages\optuna\_experimental.py:61: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  ExperimentalWarning,


{'n_estimators': 444, 'learning_rate': 0.022747204170935315, 'lambda_l1': 0.39663564159126696, 'lambda_l2': 0.5266360183584486, 'num_leaves': 11, 'feature_fraction': 0.8117818483929862, 'bagging_fraction': 0.73843817072927, 'bagging_freq': 2, 'min_child_samples': 75}
[I 2020-06-08 00:09:14,879] Finished trial#0 with value: 0.5251841272076239 with parameters: {'n_estimators': 444, 'learning_rate': 0.022747204170935315, 'lambda_l1': 0.39663564159126696, 'lambda_l2': 0.5266360183584486, 'num_leaves': 11, 'feature_fraction': 0.8117818483929862, 'bagging_fraction': 0.73843817072927, 'bagging_freq': 2, 'min_child_samples': 75}. Best is trial#0 with value: 0.5251841272076239.
{'n_estimators': 424, 'learning_rate': 0.038034945056526645, 'lambda_l1': 2.8247087122296067e-05, 'lambda_l2': 0.13351354975517513, 'num_leaves': 89, 'feature_fraction': 0.7399885861875286, 'bagging_fraction': 0.739278479610083, 'bagging_freq': 2, 'min_child_samples': 82}



KeyboardInterrupt: 

In [6]:
parameters=pd.read_csv('LGBMClassifier_grid_results.csv')
best_params=get_best_params(parameters,'score')
best_params
clf=LGBMClassifier(n_jobs=-1,**best_params)
df= pd.read_csv('ensemble_predictions.csv')
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
columns=df.drop(columns=['target','training','listing_id']).columns
training='training'
label='target'
print('modeling')
test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf )



modeling


In [11]:
best_params={'n_estimators': 444, 'learning_rate': 0.022747204170935315, 'lambda_l1': 0.39663564159126696, 'lambda_l2': 0.5266360183584486, 'num_leaves': 11, 'feature_fraction': 0.8117818483929862, 'bagging_fraction': 0.73843817072927, 'bagging_freq': 2, 'min_child_samples': 75}
clf=LGBMClassifier(n_jobs=-1,random_state=0,**best_params)
test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf )

In [12]:
lloss

0.5251841272076239

In [7]:
lloss

0.5302228421667791

In [ ]:
model_id=clf.__class__.__name__+'Ensemble'+model_id+'_'+str(round(lloss,5))

In [5]:
best_params

{'n_estimators': 972,
 'learning_rate': 0.01848416124154692,
 'lambda_l1': 1.1948456836604695,
 'lambda_l2': 8.364115989935963e-07,
 'num_leaves': 34,
 'feature_fraction': 0.7671799905376775,
 'bagging_fraction': 0.7281188949960176,
 'bagging_freq': 2,
 'min_child_samples': 88}

In [ ]:
save_csv(model_id,test_pred,yoof,clf,df_test['listing_id'],df_train['listing_id'])

In [18]:
#best_params=get_best_params(parameters,'score')
best_params={'n_estimators': 444, 'learning_rate': 0.022747204170935315, 'lambda_l1': 0.39663564159126696, 'lambda_l2': 0.5266360183584486, 'num_leaves': 11, 'feature_fraction': 0.8117818483929862, 'bagging_fraction': 0.73843817072927, 'bagging_freq': 2, 'min_child_samples': 75}
best_params
clf=LGBMClassifier(n_jobs=-1,random_state=0,**best_params)
df= pd.read_csv('ensemble_predictions_v1.csv')
columns=df.drop(columns=['target','training','listing_id']).columns
training='training'
label='target'
test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf )
lloss

0.5264353194248115

In [17]:
lloss

0.5323436013029318

In [ ]:
model_id=clf.__class__.__name__+'Ensemble'+model_id+'_'+str(round(lloss,5))

In [21]:
import functools
import optuna
from hashlib import md5
from optuna.samplers import TPESampler
df= pd.read_csv('ensemble_predictions.csv')
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
columns=df.drop(columns=['target','training','listing_id']).columns
training='training'
label='target'

def objective(df,columns,training,label,df_test_listingID,df_train_listingID,trial):
    params ={
        'n_estimators':trial.suggest_int('n_estimators', 10, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.04),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 100),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.7, 0.8),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 2),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
    
    print(params)
    clf=LGBMClassifier(n_jobs=-1,random_state=0,**params)
    test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf )
    model_id=md5(str(params).encode()).hexdigest()[:6]
    model_id=clf.__class__.__name__+'_'+model_id+'_'+str(round(lloss,3))
    trial.set_user_attr('model_id',model_id)
    save_csv (model_id,test_pred,yoof,clf,df_test_listingID,df_train_listingID,dir='Thesis_predictions_ensemble/')
    return lloss

n=10
sampler=TPESampler(seed=0)
study = optuna.create_study(direction='minimize',sampler=sampler)
fn=functools.partial(objective,df,columns,training,label,df_test['listing_id'],df_train['listing_id'])
study.optimize(fn, n_trials=n,show_progress_bar=True)

x=[{**t.params,**t.user_attrs} for t in study.get_trials()]
y=[ t.value for t in study.get_trials()]
table=pd.DataFrame(x)
table['score']=y

table.to_csv(clf.__class__.__name__+'_grid_LGBM_Ensemble.csv')

C:\Users\Martin\anaconda3\lib\site-packages\optuna\_experimental.py:61: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  ExperimentalWarning,


{'n_estimators': 694, 'learning_rate': 0.022747204170935315, 'lambda_l1': 0.39663564159126696, 'lambda_l2': 0.5266360183584486, 'num_leaves': 11, 'feature_fraction': 0.8117818483929862, 'bagging_fraction': 0.73843817072927, 'bagging_freq': 2, 'min_child_samples': 75}
[I 2020-06-08 10:07:28,113] Finished trial#0 with value: 0.5255800240757177 with parameters: {'n_estimators': 694, 'learning_rate': 0.022747204170935315, 'lambda_l1': 0.39663564159126696, 'lambda_l2': 0.5266360183584486, 'num_leaves': 11, 'feature_fraction': 0.8117818483929862, 'bagging_fraction': 0.73843817072927, 'bagging_freq': 2, 'min_child_samples': 75}. Best is trial#0 with value: 0.5255800240757177.
{'n_estimators': 482, 'learning_rate': 0.038034945056526645, 'lambda_l1': 2.8247087122296067e-05, 'lambda_l2': 0.13351354975517513, 'num_leaves': 89, 'feature_fraction': 0.7399885861875286, 'bagging_fraction': 0.739278479610083, 'bagging_freq': 2, 'min_child_samples': 82}
[I 2020-06-08 10:33:13,681] Finished trial#1 with